# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [3]:
# 필요한 라이브러리를 설치합니다.
#!pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [ ]:
import os                                           # 경로·파일 처리, 환경 변수 접근
import time                                         # 시간 측정·지연 제어
import random                                       # 표준 난수 유틸리티

import timm                                         # 사전학습 비전 모델/백본 로더
import torch                                        # 텐서·자동미분·훈련 루프 핵심
import albumentations as A                          # 고성능 이미지 증강 파이프라인
import pandas as pd                                 # 표형 데이터 로드·전처리
import numpy as np                                  # 수치 연산·배열 처리
import torch.nn as nn                               # 신경망 계층·모듈 정의
from albumentations.pytorch import ToTensorV2       # NumPy→Tensor 변환 어댑터
from torch.optim import Adam                        # Adam 옵티마이저
from torchvision import transforms                  # torchvision 기본 변환(보조/호환)
from torch.utils.data import Dataset, DataLoader    # 커스텀 데이터셋·배치 로더
from PIL import Image                               # 이미지 파일 로드·변환
from tqdm import tqdm                               # 진행률 표시(progress bar)
from sklearn.metrics import accuracy_score, f1_score# 정확도·F1 평가 지표


In [ ]:
# 재현성 확보: 전역 시드 고정 및 CuDNN 설정
SEED = 42                                           # 실험 전역 시드 값

os.environ['PYTHONHASHSEED'] = str(SEED)            # 파이썬 해시 시드 고정 → dict/set 해시 일관성
random.seed(SEED)                                   # 표준 random 모듈 시드 고정
np.random.seed(SEED)                                # NumPy 시드 고정
torch.manual_seed(SEED)                             # PyTorch CPU 시드 고정
torch.cuda.manual_seed(SEED)                        # 단일 GPU 시드 고정
torch.cuda.manual_seed_all(SEED)                    # 다중 GPU(모든 디바이스) 시드 고정

torch.backends.cudnn.benchmark = True               # CuDNN 알고리즘 자동 선택(속도↑)


In [ ]:
# 커스텀 이미지 데이터셋 정의: CSV 기반 로더
class ImageDataset(Dataset):
    # 초기화: CSV 경로, 이미지 루트, 변환 파이프라인 수신
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values               # CSV 로드 → (파일명, 라벨) 배열 확보
        self.path = path                                # 이미지 루트 경로 저장
        self.transform = transform                      # Albumentations 변환 파이프라인 저장

    # 데이터셋 길이: 샘플 수 반환
    def __len__(self):
        return len(self.df) # 총 행 수 반환

    # 샘플 접근: (이미지, 라벨) 튜플 반환
    def __getitem__(self, idx):
        name, target = self.df[idx]                     # 파일명, 라벨 분리
        img = np.array(                                 # 이미지 파일 로드 후 배열 변환
            Image.open(os.path.join(self.path, name))
        )
        if self.transform:                              # 변환 파이프라인 설정 시
            img = self.transform(image=img)['image']    # 변환 적용 후 Tensor/배열 획득
        return img, target                              # 모델 입력, 라벨 반환


In [ ]:
# one epoch 학습 루프: 순전파→손실→역전파→최적화→지표 계산
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()       # 학습 모드 전환
    train_loss = 0      # 에폭 손실 누적 변수
    preds_list = []     # 예측 레이블 누적 버퍼
    targets_list = []   # 정답 레이블 누적 버퍼

    pbar = tqdm(loader) # 진행률 표시 래퍼
    
    # 배치 단위 학습/추론/기록
    for image, targets in pbar:
        image = image.to(device)            # 입력 배치를 디바이스로 이동
        targets = targets.to(device)        # 타깃을 디바이스로 이동

        model.zero_grad(set_to_none=True)   # 그래디언트 초기화(메모리 효율)

        preds = model(image)                # 순전파 수행
        loss = loss_fn(preds, targets)      # 손실 계산
        loss.backward()                     # 역전파로 기울기 계산
        optimizer.step()                    # 파라미터 업데이트

        train_loss += loss.item()           # 배치 손실 누적
        
        # 배치 예측 라벨 수집
        preds_list.extend(
            preds.argmax(dim=1).detach().cpu().numpy()
        )
        
        # 배치 정답 라벨 수집
        targets_list.extend(
            targets.detach().cpu().numpy()
        )

        # 현재 배치 손실 표시
        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)                                       # 에폭 평균 손실 계산
    train_acc = accuracy_score(targets_list, preds_list)            # 에폭 정확도 계산
    train_f1 = f1_score(targets_list, preds_list, average='macro')  # 에폭 매크로 F1 계산

    # 에폭 성능 요약 딕셔너리
    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    # 요약 결과 반환
    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [ ]:
# 디바이스 선택: CUDA 가능 시 'cuda', 아니면 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 가속 장치 자동 선택

# 데이터 경로 설정
data_path = '../data/raw/'  # 원본 데이터 루트 경로

# 모델 설정
model_name = 'resnet34'     # 사용 백본 이름 예시: 'resnet50', 'efficientnet-b0' 등

# 학습 하이퍼파라미터
img_size    = 32            # 입력 이미지 한 변 픽셀 크기
LR          = 1e-3          # 학습률
EPOCHS      = 1             # 총 학습 에폭 수
BATCH_SIZE  = 32            # 배치 크기
num_workers = 0             # DataLoader 워커 수(0=메인 프로세스; OS/환경 이슈 회피)

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [ ]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([                         # 학습용 변환 파이프라인
    A.Resize(height=img_size, width=img_size),      # 크기 조정
    A.Normalize(mean=[0.485, 0.456, 0.406],         # 채널별 정규화(mean/std)
                std=[0.229, 0.224, 0.225]),             
    ToTensorV2(),                                   # NumPy/PIL → PyTorch Tensor 변환
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([                         # 평가/추론용 변환 파이프라인
    A.Resize(height=img_size, width=img_size),      # 크기 조정
    A.Normalize(mean=[0.485, 0.456, 0.406],         # 채널별 정규화(mean/std)
                std=[0.229, 0.224, 0.225]),
    ToTensorV2(),                                   # Tensor 변환
])


In [ ]:
# Dataset 정의
# 학습용 데이터셋 인스턴스
trn_dataset = ImageDataset(
    "../data/raw/train.csv",                # 학습 CSV 경로
    "../data/raw/train/",                   # 학습 이미지 루트
    transform=trn_transform                 # 학습용 변환 파이프라인
)

# 평가/추론용 데이터셋 인스턴스
tst_dataset = ImageDataset(
    "../data/raw/sample_submission.csv",    # 제출 샘플 CSV 또는 테스트 ID 목록
    "../data/raw/test/",                    # 테스트 이미지 루트
    transform=tst_transform                 # 평가/추론용 변환 파이프라인
)

In [ ]:
# DataLoader 정의
# 학습용 배치 로더
trn_loader = DataLoader(
    trn_dataset,            # 학습 데이터셋 객체
    batch_size=BATCH_SIZE,  # 배치 크기
    shuffle=True,           # 에폭마다 섞기
    num_workers=num_workers,# 로딩 워커 수
    pin_memory=True,        # 핀 메모리 사용(CUDA 전송 최적화)
    drop_last=False         # 마지막 불완전 배치 유지
)

# 평가/추론용 배치 로더
tst_loader = DataLoader(
    tst_dataset,            # 평가/추론 데이터셋 객체
    batch_size=BATCH_SIZE,  # 배치 크기
    shuffle=False,          # 순서 유지(섞지 않음)
    num_workers=0,          # 로딩 워커 수(평가 고정)
    pin_memory=True         # 핀 메모리 사용
)


## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [ ]:
# load model
# timm 라이브러리를 통해 사전학습 모델 로드
model = timm.create_model(
    model_name,        # 모델 이름 (예: resnet34)
    pretrained=True,   # ImageNet 등으로 학습된 가중치 사용
    num_classes=17     # 출력 클래스 수 지정
).to(device)           # 지정된 디바이스(CPU/GPU)로 이동

# 손실 함수 정의 (다중 클래스 분류용)
loss_fn = nn.CrossEntropyLoss()

# 최적화 알고리즘 정의 (Adam)
optimizer = Adam(model.parameters(), lr=LR)  # 모델 파라미터와 학습률 설정

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [ ]:
# 학습 루프 실행
for epoch in range(EPOCHS):  
    # 한 에폭 학습 수행
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)  
    ret['epoch'] = epoch  # 현재 에폭 번호 기록

    log = ""  # 로그 문자열 초기화
    # 결과 딕셔너리 항목 순회
    for k, v in ret.items():  
        log += f"{k}: {v:.4f}\n"  # key와 값을 문자열로 누적
    print(log)  # 에폭 결과 출력

Loss: 2.2397: 100%|██████████| 50/50 [00:04<00:00, 11.58it/s]

train_loss: 2.4603
train_acc: 0.2631
train_f1: 0.2253
epoch: 0.0000



# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
# 예측 결과 저장 리스트 초기화
preds_list = []

# 모델 평가 모드 전환
model.eval()
# 테스트 로더 순회
for image, _ in tqdm(tst_loader):
    image = image.to(device)  # 입력 이미지를 디바이스로 이동

    # 기울기 계산 비활성화(추론 전용)
    with torch.no_grad():
        preds = model(image)  # 모델 추론 수행
    # 예측 라벨을 CPU NumPy 배열로 변환 후 저장
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:06<00:00, 15.71it/s]


In [ ]:
# 예측 결과 DataFrame 생성
pred_df = pd.DataFrame(                              # DataFrame 초기화
    tst_dataset.df,                                  # 테스트 데이터셋 원본 ID/target
    columns=['ID', 'target']                         # 컬럼명 지정
)
pred_df['target'] = preds_list                       # target 컬럼을 예측 결과로 갱신

In [ ]:
# 제출 샘플 CSV 로드
sample_submission_df = pd.read_csv("../data/raw/sample_submission.csv")

# 예측 결과 ID와 제출 샘플 ID가 동일한지 검증
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [ ]:
# 예측 결과를 CSV로 저장
pred_df.to_csv(
    "../submissions/20250903/baseline_code_pred.csv",  # 저장 경로
    index=False                                        # 인덱스 제외
)

In [ ]:
# 예측 결과 확인: 상위 5개 행 출력
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,11
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,10
4,00901f504008d884.jpg,16
